# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os
from urllib.parse import urlencode

# Import API key
from api_keys import g_key


In [5]:
# Intput File (CSV)
# input_data_file = "output_data/cities.csv"
input_data_file = "../WeatherPy/output_data/my_cities.csv"


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
csvDF = pd.read_csv(input_data_file)
csvDF.head(10)


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_ID
0,Solnechnyy,50.7214,136.6319,0.25,88,0,1.99,RU,1615124318,0
1,Saldanha,-33.0117,17.9442,71.60,64,100,17.27,ZA,1615124318,1
2,Cherskiy,68.7500,161.3000,-20.51,88,93,6.44,RU,1615124318,2
3,Busselton,-33.6500,115.3333,67.24,84,79,1.99,AU,1615124318,3
4,Punta Arenas,-53.1500,-70.9167,42.39,87,75,17.27,CL,1615124318,4
5,Castro,-24.7911,-50.0119,69.55,87,75,3.22,BR,1615124249,5
6,Karlshamn,56.1706,14.8619,40.10,38,0,17.27,SE,1615124318,6
7,Bariloche,-41.1456,-71.3082,62.60,59,40,23.02,AR,1615124319,7
8,Cape Town,-33.9258,18.4232,66.49,68,75,20.71,ZA,1615124319,8
9,Mataura,-46.1927,168.8643,51.01,69,85,15.99,NZ,1615124319,9


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)


In [8]:
locations = csvDF[['Lat', 'Lon']].astype(float)
humidity = csvDF['Humidity'].astype(float)

In [9]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=90,
                                 point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# check DF
csvDF.head(3)

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_ID
0,Solnechnyy,50.7214,136.6319,0.25,88,0,1.99,RU,1615124318,0
1,Saldanha,-33.0117,17.9442,71.60,64,100,17.27,ZA,1615124318,1
2,Cherskiy,68.7500,161.3000,-20.51,88,93,6.44,RU,1615124318,2


In [11]:
# Set Ideal weather condition variables
lowerIdealTemp = 68
upperIdealTemp = 80
idealWindspeed = 10
cloudiness = 0


In [12]:
idealWeatherDF = csvDF.loc[(csvDF['Max Temp'] >= lowerIdealTemp) & \
                           (csvDF['Max Temp'] < upperIdealTemp) & (csvDF['Wind Speed'] < idealWindspeed) & \
                           (csvDF['Cloudiness'] == cloudiness)]
print(len(idealWeatherDF))
idealWeatherDF.describe()

23


,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Date,City_ID
count,23.000000,23.000000,23.000000,23.000000,23.0,23.000000,2.300000e+01,23.000000
mean,-3.958696,11.857109,74.695217,51.608696,0.0,4.820870,1.615124e+09,270.782609
std,26.248691,77.310974,3.236682,23.721373,0.0,2.070244,7.235423e+01,158.442693
min,-45.866700,-102.200000,68.000000,18.000000,0.0,1.810000,1.615124e+09,27.000000
25%,-29.041800,-60.916650,73.075000,31.000000,0.0,3.500000,1.615124e+09,146.500000
50%,14.300000,26.891000,75.200000,50.000000,0.0,4.610000,1.615124e+09,262.000000
75%,18.533200,79.916700,76.360000,75.500000,0.0,5.845000,1.615124e+09,419.500000
max,26.283300,148.233300,79.470000,94.000000,0.0,9.910000,1.615124e+09,517.000000


In [13]:
# drop na values: 
idealWeatherDF = idealWeatherDF.dropna()
print(len(idealWeatherDF))
idealWeatherDF.head(4)

22


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_ID
27,Tura,25.5198,90.2201,68.63,33,0,2.91,IN,1615124321,27
51,Port Alfred,-33.5906,26.8910,75.00,78,0,4.00,ZA,1615124325,51
53,Neuquén,-38.9516,-68.0591,73.40,49,0,4.61,AR,1615124211,53
80,Rocha,-34.4833,-54.3333,75.51,56,0,4.43,UY,1615124329,80


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = idealWeatherDF.copy()

In [15]:
hotel_df.rename(columns={'Lon': 'Lng'}, inplace=True)
hotel_df.head(3)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_ID
27,Tura,25.5198,90.2201,68.63,33,0,2.91,IN,1615124321,27
51,Port Alfred,-33.5906,26.8910,75.00,78,0,4.00,ZA,1615124325,51
53,Neuquén,-38.9516,-68.0591,73.40,49,0,4.61,AR,1615124211,53


In [16]:
# get a list of the coordinates as list
coordinatesList = hotel_df[['City','Lat', 'Lng']].values.tolist()
coordinatesList[0:3]

[['Tura', 25.5198, 90.2201],
 ['Port Alfred', -33.5906, 26.891],
 ['Neuquén', -38.9516, -68.0591]]

### Nearby hotels search

In [17]:
# nearby hotels search
target_coordinates = coordinatesList
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"
hotelsDict = []

# Iterate through the coodinatesList and add params:
for coord in target_coordinates: 
    params = {
        "location": f'{coord[1]}, {coord[2]}',
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    response = requests.get(base_url, params=params).json()
    hotelsDict.append((coord[0], response['results']))
    

In [18]:
# Get the hotel names and geo coord in list
hotelsList = []
for i in hotelsDict:
    try:
        hotelsList.append([i[0], i[1][0].get('name'), i[1][0].get('geometry').get('location').get('lat'), i[1][0].get('geometry').get('location').get('lng')])
    except IndexError as e:
        hotelsList.append([i[0], 'no record'])

In [19]:
nearByhotels = pd.DataFrame(hotelsList, columns=['CityName', 'Hotel Name', 'Lat', 'Lng'])
nearByhotels.head(3)

,CityName,Hotel Name,Lat,Lng
0,Tura,Hotel Polo Orchid,25.514468,90.192206
1,Port Alfred,Royal St Andrews Hotel,-33.603178,26.886978
2,Neuquén,Hotel Huemul,-38.958652,-68.052328


In [20]:
hotel_df.head(4)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_ID
27,Tura,25.5198,90.2201,68.63,33,0,2.91,IN,1615124321,27
51,Port Alfred,-33.5906,26.8910,75.00,78,0,4.00,ZA,1615124325,51
53,Neuquén,-38.9516,-68.0591,73.40,49,0,4.61,AR,1615124211,53
80,Rocha,-34.4833,-54.3333,75.51,56,0,4.43,UY,1615124329,80


In [21]:
# Merge the two dataframes by city and city name
hotel_df = hotel_df.merge(nearByhotels, left_on='City', right_on='CityName').dropna()

In [22]:
hotel_df.head(4)

,City,Lat_x,Lng_x,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_ID,CityName,Hotel Name,Lat_y,Lng_y
0,Tura,25.5198,90.2201,68.63,33,0,2.91,IN,1615124321,27,Tura,Hotel Polo Orchid,25.514468,90.192206
1,Port Alfred,-33.5906,26.8910,75.00,78,0,4.00,ZA,1615124325,51,Port Alfred,Royal St Andrews Hotel,-33.603178,26.886978
2,Neuquén,-38.9516,-68.0591,73.40,49,0,4.61,AR,1615124211,53,Neuquén,Hotel Huemul,-38.958652,-68.052328
4,Rāichūr,16.2000,77.3667,79.27,21,0,1.81,IN,1615124334,105,Rāichūr,Hotel SAK,16.186175,77.345347


In [23]:
hotel_info = hotel_df[['Hotel Name', 'Lat_y', 'Lng_y']].values.tolist()

In [24]:
hotel_locations = hotel_df[['Lat_y', 'Lng_y']].astype(float)

In [25]:
fig = gmaps.figure()
hotel_layer = gmaps.marker_layer(hotel_locations)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat_y", "Lng_y"]]


In [27]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))